# Script para a inserção das entidades PEPs no banco


In [ ]:
import pandas as pd
import os, sys, django, neomodel, dotenv, wikipedia
from datetime import datetime

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(BASE_DIR)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "tcc.settings")
django.setup()
dotenv.load_dotenv()

from myapi.models import Pessoa, Organizacao, Cargo

In [ ]:
%run wiki_data.ipynb

In [ ]:
def insere_pep(nome, cpf, nascimento, cnpj) -> Pessoa:
    try:
        pep = Pessoa(nome=nome, 
                     cpf=cpf, 
                     data_nascimento=pd.to_datetime(nascimento, format='%d/%m/%Y', errors='coerce'), 
                     cnpj=cnpj).save()
        print(f'Pessoa {nome} inserida com sucesso.')
        return pep
    except Exception as e:
        print(f"Erro ao inserir PEP: {e}")
    return None

def insere_organizacao(nome, cnpj) -> Organizacao:
    try:
        org = Organizacao(nome=nome, cnpj=cnpj).save()
        print(f'Organização {nome} inserida com sucesso.')
        return org
    except Exception as e:
        print(f"Erro ao inserir organização: {e}")
    return None

def insere_cargo(nome) -> Cargo:
    try:
        cargo = Cargo(nome=nome).save()
        print(f'Cargo {nome} inserido com sucesso.')
        return cargo
    except Exception as e:
        print(f"Erro ao inserir cargo: {e}")
    return None

def relaciona_cargo_pessoa_organizacao(cargo: Cargo, pessoa: Pessoa, org:Organizacao, papel, inicio, fim) -> None:
    try:
        pessoa.cargo.connect(cargo, {'papel': papel,
                                    'data_inicio': pd.to_datetime(inicio, format='%d/%m/%Y', errors='coerce'), 
                                    'data_fim': pd.to_datetime(fim, format='%d/%m/%Y', errors='coerce') if not pd.isna(fim) else None,
                                    'grau_precisao': 5})
        cargo.pertence.connect(org)
        print(f'Relação entre {pessoa.nome}, {cargo.nome} e {org.nome} criada com sucesso.')
    except Exception as e:
        print(f"Erro ao criar relação: {e}")
        return

In [ ]:
csv_path = os.path.join(BASE_DIR, 'pipelines', 'data', '202507_PEP.csv')
if os.path.exists(csv_path):
	df = pd.read_csv(csv_path, encoding='latin1', dtype=str, on_bad_lines='warn', sep=';').fillna('')
else:
	print(f"Arquivo não encontrado: {csv_path}")

In [ ]:
for index, row in df.iterrows():
    nome = row['Nome_PEP']
    cpf = row['CPF']
    cnpj = None
    org_nome = row['Nome_Órgão']
    org_cnpj = None
    cargo_nome = row['Descrição_Função']
    inicio = row['Data_Início_Exercício']
    fim = row['Data_Fim_Exercício']
    try:
        pagina_wiki = wikipedia.page(title=nome)
    except Exception as e:
        print(e)
        continue

    try:
        pessoa = Pessoa.nodes.get(nome=nome, cpf=cpf)
    except neomodel.DoesNotExist as e:
        pessoa = insere_pep(nome, cpf, nascimento, cnpj)

    try:
        org = Organizacao.nodes.get(nome=org_nome)
    except neomodel.DoesNotExist as e:
        org = insere_organizacao(org_nome, org_cnpj)

    try:
        cargo = Cargo.nodes.get(nome=cargo_nome)
    except neomodel.DoesNotExist as e:
        cargo = insere_cargo(cargo_nome)

    if pessoa != None and org != None and cargo != None:
        relaciona_cargo_pessoa_organizacao(cargo, pessoa, org, papel=cargo_nome, inicio=inicio, fim=fim)
    
    conjuge = find_personal_info('Côjunge',pagina_wiki)
    filhos = find_personal_info('Filhos(as)',pagina_wiki)
    nascimento = find_personal_info('Nascimento',pagina_wiki)
    data_nascimento = nascimento[0] + ' de ' + nascimento[1]
    data_nascimento = datetime.strptime(data_nascimento, "%d de %B de %Y")
    progenitores = find_personal_info('Progenitores',pagina_wiki)